# Doctests for jupyter notebook

Handy way to process the doctests in jupyter notebooks (.ipynb) containing Python functions.
The script converts an .ipynb to a string format (basically a .py file), loads them as modules, and runs doctest on them.
To run it in the console, do:

    python -m pytest --verbose --disable-warnings --nbval test_ipynb.ipynb

The script should tell you which ipynb file's doctests has failed (e.g. srgan_train.ipynb).
You can then open up this very jupyter notebook to debug and inspect the situation further.

In [1]:
import ast
import doctest
import nbconvert
import nbformat
import os
import sys
import types

def _doctest_ipynb(path:str):
    """
    First converts .ipynb to a temporary .py file
    Then it runs doctest on the temp .py file
    """
    assert(path.endswith('.ipynb'))
    basename, _ = os.path.splitext(path)
    
    #read ipynb file and get the text
    with open(path) as ipynb_file:
        nb = nbformat.reads(s=ipynb_file.read(), as_version=nbformat.NO_CONVERT)
    assert(isinstance(nb, nbformat.notebooknode.NotebookNode))
    
    #convert the .ipynb text to a string .py format
    pyexporter = nbconvert.PythonExporter()
    source, meta = pyexporter.from_notebook_node(nb=nb)
    assert(isinstance(source, str))
    
    #parse the .py string to pick out only 'import' and 'def function's
    parsed_code = ast.parse(source=source)
    for node in parsed_code.body[:]:
        if node.__class__ not in [ast.FunctionDef, ast.Import, ast.ImportFrom]:
            parsed_code.body.remove(node)
    assert(len(parsed_code.body) > 0)
    
    #import modules from the parsed .py string
    module = types.ModuleType(basename)
    code = compile(source=parsed_code, filename=f'{basename}.py', mode='exec')
    exec(code, module.__dict__)
    
    num_failures, num_attempted = doctest.testmod(m=module, verbose=True)
    if num_failures > 0:
        sys.exit(num_failures)

In [2]:
_doctest_ipynb('data_prep.ipynb')

Trying:
    download_to_path(path="highres/2017_Antarctica_Basler.csv", url="https://data.cresis.ku.edu/data/rds/2017_Antarctica_Basler/csv_good/2017_Antarctica_Basler.csv").headers['Content-Length']
Expecting:
    '64'
ok
3 items had no tests:
    data_prep
    data_prep.get_window_bounds
    data_prep.selective_tile
1 items passed all tests:
   1 tests in data_prep.download_to_path
1 tests in 4 items.
1 passed and 0 failed.
Test passed.


In [3]:
_doctest_ipynb('srgan_train.ipynb')

Using TensorFlow backend.


Trying:
    metrics = {"generator_network": 'mse', "discriminator_network": 'accuracy'}
Expecting nothing
ok
Trying:
    models = compile_srgan_model(g_network=generator_network(), d_network=discriminator_network(), metrics=metrics)
Expecting nothing
ok
Trying:
    models['discriminator_model'].trainable
Expecting:
    True
ok
Trying:
    models['srgan_model'].get_layer(name='generator_network').trainable
Expecting:
    True
ok
Trying:
    models['srgan_model'].get_layer(name='discriminator_network').trainable
Expecting:
    False
ok
Trying:
    models['srgan_model'].count_params()
Expecting:
    8571362
ok
Trying:
    discriminator_network().input_shape
Expecting:
    (None, 32, 32, 1)
ok
Trying:
    discriminator_network().output_shape
Expecting:
    (None, 1)
ok
Trying:
    discriminator_network().count_params()
Expecting:
    6828033
ok
Trying:
    generator_network().input_shape
Expecting:
    [(None, 8, 8, 1), (None, 40, 40, 1), (None, 16, 16, 1)]
ok
Trying:
    generator_network